# Sentence Classification with Huggingface-Trainer

Creators: Luca Moroni

Official Reference: https://huggingface.co/blog/sentiment-analysis-python

To face this notebook you must read and understand well the "NLP Notebook #5 - Transformers".

In this EXTRA notebook we will see the usage of Huggingface Trainer, a tested pipeline that led us to use less code and concentrate on other aspects than create the wheel again every times (e.g. training for loop, early stopping, ...).


Let's go through the code!

In [ ]:
!pip install accelerate -U
!pip install datasets

In [ ]:
import torch
import numpy as np
import pandas as pd
from typing import Dict
import torch
from datasets import load_dataset
from transformers import DataCollatorWithPadding

from datasets import load_dataset
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    EvalPrediction,
    Trainer,
    TrainingArguments,
    set_seed,
)


### Model Parameters
# we will use with Distil-BERT
language_model_name = "distilbert-base-uncased"

### Training Argurments

# this GPU should be enough for this task to handle 32 samples per batch
batch_size = 32

# optim
learning_rate = 1e-4
weight_decay = 0.001 # we could use e.g. 0.01 in case of very low and very high amount of data for regularization

# training
epochs = 1
device = "cuda" if torch.cuda.is_available() else "cpu"


set_seed(42)

: 

In this example we will use **Sentiment Tree Bank corpus (SST2)**, one of the main benchmark for binary sentiment analysis task.

The dataset if freely available through Huggingface Datasets.

In [ ]:
# load our dataset
sst2_dataset = load_dataset("stanfordnlp/sst2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
## Let's see an example...
print(f"Sentence: {sst2_dataset['train']['sentence'][12]}")
print(f"Sentiment: {'Positive' if sst2_dataset['train']['label'][12] == 1 else 'Negative'}")

Sentence: the part where nothing 's happening , 
Sentiment: Negative


In [ ]:
## The structure of the huggingface dataset.
## Here the test set cannot be used since is a blind test set, so there aren't the gold labels.
sst2_dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 1821
    })
})

### Metric Definition

Looking only at cross entropy loss cannot allow us to understand effectivelly the real capabilities of our NLP model. So let's define a standard method to compute:

- **Accuracy** metric
- **F1** metric

In [ ]:
from datasets import load_metric

# Metrics

def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

## The Model

We will rely on the Huggingface **AutoModelForSequenceClassification** class from huggingface repository. This is a wrapper for encoder-only models, that allow us to simply create a model suitable to solve a classification task over textual sentences.

### Sentence Classification

In the previous notebook you saw how to train a token level classifier to solve NER task, learning a MLP over each tokens of the input sentence. In the setting of sentence level classification the standard approaches apply a MLP over the [CLS] token's embedding of the last encoder layer. This is because the [CLS] token contains a lot of information about the **semantic and syntactitc** structure of the input sentence.

![alt text](https://jalammar.github.io/images/bert-classifier.png "Sentence Classification")

In [ ]:
## Initialize the model
model = AutoModelForSequenceClassification.from_pretrained(language_model_name,
                                                                   ignore_mismatched_sizes=True,
                                                                   output_attentions=False, output_hidden_states=False,
                                                                   num_labels=2) # number of the classes

tokenizer = AutoTokenizer.from_pretrained(language_model_name)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding=True, truncation=True)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenize the dataset ...
print("Tokenize the dataset ...")
tokenized_datasets_sst2 = sst2_dataset.map(tokenize_function, batched=True)

Tokenize the dataset ...


Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

## Model Training

To train a transformer model you can rely on the **Trainer** class of Huggingface (https://huggingface.co/docs/transformers/main_classes/trainer).

The Trainer class allows you to save many lines of code, and makes your code much more readable.

To initialize the Trainer class you have to define a **TrainerArguments** object.

In [ ]:
training_args = TrainingArguments(
    output_dir="training_dir",                    # output directory [Mandatory]
    num_train_epochs=epochs,                      # total number of training epochs
    per_device_train_batch_size=batch_size,       # batch size per device during training
    warmup_steps=500,                             # number of warmup steps for learning rate scheduler
    weight_decay=weight_decay,                    # strength of weight decay
    save_strategy="no",
    learning_rate=learning_rate                   # learning rate
)

In [ ]:
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_datasets_sst2["train"],
   eval_dataset=tokenized_datasets_sst2["validation"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [ ]:
# Let's Train ...
trainer.train()

Step,Training Loss
500,0.362700
1000,0.244700
1500,0.201300
2000,0.172500


TrainOutput(global_step=2105, training_loss=0.24076004481372243, metrics={'train_runtime': 392.5057, 'train_samples_per_second': 171.587, 'train_steps_per_second': 5.363, 'total_flos': 1139506554896892.0, 'train_loss': 0.24076004481372243, 'epoch': 1.0})

In [ ]:
# Evaluate the model ...
trainer.evaluate()

<ipython-input-8-6b5f7f4e622d>:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the re

{'eval_loss': 0.26060113310813904,
 'eval_accuracy': 0.9025229357798165,
 'eval_f1': 0.9054505005561735,
 'eval_runtime': 3.9436,
 'eval_samples_per_second': 221.118,
 'eval_steps_per_second': 27.64,
 'epoch': 1.0}